In [ ]:
# Step 1: Install packages
!pip install -q torch torchvision tqdm matplotlib numpy wandb

import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using: {DEVICE}")

In [ ]:
# Step 2: Login to WandB
import wandb
wandb.login()
print("WandB logged in!")

In [ ]:
# Step 3: Clone repo (REPLACE YOUR_USERNAME)
!git clone https://github.com/99VICKY99/Fed-Audit-GAN.git
%cd Fed-Audit-GAN
!git checkout strict-4-phase
!git branch --show-current

In [ ]:
# Step 4: Complete Training Script with WandB
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
import numpy as np
import copy
from tqdm.notebook import tqdm
import os
import pickle
import wandb

class CNN(nn.Module):
    def __init__(self, num_classes=10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        return self.fc2(x)

class FairnessGenerator(nn.Module):
    def __init__(self, latent_dim=100, num_classes=10, img_shape=(1, 28, 28)):
        super().__init__()
        self.latent_dim = latent_dim
        self.num_classes = num_classes
        self.img_shape = img_shape
        self.label_emb = nn.Embedding(num_classes, latent_dim)
        self.init_size = img_shape[1] // 4
        self.l1 = nn.Linear(latent_dim * 2, 128 * self.init_size ** 2)
        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128), nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, 1, 1), nn.BatchNorm2d(128), nn.LeakyReLU(0.2),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, 1, 1), nn.BatchNorm2d(64), nn.LeakyReLU(0.2),
            nn.Conv2d(64, img_shape[0], 3, 1, 1), nn.Tanh())
        self.delta_net = nn.Sequential(
            nn.Linear(latent_dim, 256), nn.ReLU(),
            nn.Linear(256, int(np.prod(img_shape))), nn.Tanh())
        self.delta_scale = 0.1

    def forward(self, z, labels):
        gen_input = torch.cat([z, self.label_emb(labels)], dim=1)
        out = self.l1(gen_input).view(-1, 128, self.init_size, self.init_size)
        x = self.conv_blocks(out)
        delta = self.delta_net(z).view(-1, *self.img_shape) * self.delta_scale
        return x, torch.clamp(x + delta, -1, 1)

class Discriminator(nn.Module):
    def __init__(self, num_classes=10, img_shape=(1, 28, 28)):
        super().__init__()
        self.num_classes = num_classes
        self.img_shape = img_shape
        self.label_emb = nn.Embedding(num_classes, num_classes)
        self.conv = nn.Sequential(
            nn.Conv2d(img_shape[0] + num_classes, 16, 3, 2, 1), nn.LeakyReLU(0.2),
            nn.Conv2d(16, 32, 3, 2, 1), nn.BatchNorm2d(32), nn.LeakyReLU(0.2),
            nn.Conv2d(32, 64, 3, 2, 1), nn.BatchNorm2d(64), nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, 3, 2, 1), nn.BatchNorm2d(128), nn.LeakyReLU(0.2))
        self.fc = nn.Sequential(nn.Linear(128 * 4, 1), nn.Sigmoid())

    def forward(self, img, labels):
        label_map = self.label_emb(labels).view(-1, self.num_classes, 1, 1)
        label_map = label_map.expand(-1, -1, self.img_shape[1], self.img_shape[2])
        out = self.conv(torch.cat([img, label_map], dim=1))
        return self.fc(out.view(out.size(0), -1))

def train_gan(G, D, model, loader, epochs=30, device='cuda', l1=1.0, l2=1.0):
    G, D, model = G.to(device), D.to(device), model.to(device)
    model.eval()
    opt_G = optim.Adam(G.parameters(), lr=0.0002, betas=(0.5, 0.999))
    opt_D = optim.Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))
    bce = nn.BCELoss()
    for _ in range(epochs):
        for imgs, labels in loader:
            bs = imgs.size(0)
            real, fake_t = torch.ones(bs, 1, device=device), torch.zeros(bs, 1, device=device)
            imgs, labels = imgs.to(device), labels.to(device)
            z = torch.randn(bs, G.latent_dim, device=device)
            gl = torch.randint(0, G.num_classes, (bs,), device=device)
            x, xp = G(z, gl)
            with torch.no_grad():
                px, pxp = model(x), model(xp)
            t1 = -torch.mean((px - pxp) ** 2)
            t2 = l1 * torch.mean((x - xp) ** 2)
            t3 = l2 * (bce(D(x, gl), real) + bce(D(xp, gl), real)) / 2
            opt_G.zero_grad(); (t1 + t2 + t3).backward(); opt_G.step()
            x, xp = G(z, gl)
            d_loss = (bce(D(imgs, labels), real) + bce(D(x.detach(), gl), fake_t) + bce(D(xp.detach(), gl), fake_t)) / 3
            opt_D.zero_grad(); d_loss.backward(); opt_D.step()
    return G, D

def compute_bias(model, x, xp, device):
    model.eval()
    with torch.no_grad():
        return torch.abs(model(x.to(device)) - model(xp.to(device))).sum(1).mean().item()

def partition_data(dataset, n):
    idx = np.argsort([dataset[i][1] for i in range(len(dataset))])
    shards = np.array_split(idx, n * 2)
    np.random.shuffle(shards)
    return [np.concatenate([shards[2*i], shards[2*i+1]]) for i in range(n)]

def evaluate(model, loader, device):
    model.eval()
    correct = sum((model(d.to(device)).argmax(1) == t.to(device)).sum().item() for d, t in loader)
    return 100 * correct / sum(len(t) for _, t in loader)

# Config
N_ROUNDS = 10
N_CLIENTS = 5
GAMMA = 2.0
N_GAN_EPOCHS = 20
N_PROBES = 300
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Init WandB
wandb.init(project="fed-audit-gan", name=f"colab_gamma{GAMMA}_clients{N_CLIENTS}", config={
    "n_rounds": N_ROUNDS, "n_clients": N_CLIENTS, "gamma": GAMMA, "device": DEVICE})

# Data
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
train_data = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST('./data', train=False, download=True, transform=transform)
client_idx = partition_data(train_data, N_CLIENTS)
test_loader = DataLoader(test_data, batch_size=64)
val_loader = DataLoader(Subset(train_data, np.random.choice(len(train_data), 1000)), batch_size=32)

# Model
global_model = CNN().to(DEVICE)
history = {'acc': [], 'bias': []}

print(f"Training: {N_ROUNDS} rounds, {N_CLIENTS} clients, gamma={GAMMA}")
for r in range(N_ROUNDS):
    print(f"\n=== Round {r+1}/{N_ROUNDS} ===")
    
    # Phase 1: Client training
    updates = []
    for c in tqdm(range(N_CLIENTS), desc="Phase 1"):
        loader = DataLoader(Subset(train_data, client_idx[c]), batch_size=32, shuffle=True)
        local = copy.deepcopy(global_model)
        before = copy.deepcopy(global_model.state_dict())
        opt = optim.SGD(local.parameters(), lr=0.01)
        local.train()
        for _ in range(3):
            for d, t in loader:
                opt.zero_grad()
                F.cross_entropy(local(d.to(DEVICE)), t.to(DEVICE)).backward()
                opt.step()
        updates.append({k: local.state_dict()[k] - before[k] for k in before})
    
    # Phase 2: GAN
    print("Phase 2: GAN training")
    G = FairnessGenerator()
    D = Discriminator()
    G, D = train_gan(G, D, global_model, val_loader, epochs=N_GAN_EPOCHS, device=DEVICE)
    G.eval()
    with torch.no_grad():
        z = torch.randn(N_PROBES, G.latent_dim, device=DEVICE)
        lbl = torch.randint(0, 10, (N_PROBES,), device=DEVICE)
        x, xp = G(z, lbl)
    
    # Phase 3: Scoring
    B_base = compute_bias(global_model, x, xp, DEVICE)
    print(f"Phase 3: B_base={B_base:.4f}")
    S = []
    for i, upd in enumerate(updates):
        hyp = copy.deepcopy(global_model)
        sd = hyp.state_dict()
        for k in sd: sd[k] = sd[k] + upd[k]
        hyp.load_state_dict(sd)
        B_i = compute_bias(hyp, x, xp, DEVICE)
        S.append(B_base - B_i)
        print(f"  Client {i}: S={S[-1]:+.4f}")
    
    # Phase 4: Aggregation
    alpha = F.softmax(torch.tensor(S) * GAMMA, dim=0).tolist()
    print(f"Phase 4: alpha={[f'{a:.3f}' for a in alpha]}")
    sd = global_model.state_dict()
    for k in sd:
        sd[k] = sd[k] + sum(a * u[k] for a, u in zip(alpha, updates))
    global_model.load_state_dict(sd)
    
    acc = evaluate(global_model, test_loader, DEVICE)
    history['acc'].append(acc)
    history['bias'].append(B_base)
    wandb.log({'round': r+1, 'accuracy': acc, 'bias': B_base, 'avg_S': np.mean(S)})
    print(f"Accuracy: {acc:.2f}%")

print(f"\nFinal: {history['acc'][-1]:.2f}%")
wandb.finish()

In [ ]:
# ============================================================
# BASELINE: Standard FedAvg (No Fairness-Aware Aggregation)
# ============================================================
# This runs the SAME setup but with uniform weights (γ = 0 equivalent)
# Formula: α_i = 1/N for all clients (standard FedAvg)

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
import numpy as np
import copy
from tqdm.notebook import tqdm
import wandb

# Reuse same CNN, Generator, Discriminator from above
# Just redefine to ensure they're available

class CNN_Baseline(nn.Module):
    def __init__(self, num_classes=10):
        super(CNN_Baseline, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        return self.fc2(x)

def evaluate_baseline(model, loader, device):
    model.eval()
    correct = sum((model(d.to(device)).argmax(1) == t.to(device)).sum().item() for d, t in loader)
    return 100 * correct / sum(len(t) for _, t in loader)

def compute_bias_baseline(model, x, xp, device):
    model.eval()
    with torch.no_grad():
        return torch.abs(model(x.to(device)) - model(xp.to(device))).sum(1).mean().item()

# Same config as Fed-Audit-GAN for fair comparison
N_ROUNDS_BASE = N_ROUNDS  # Use same as above
N_CLIENTS_BASE = N_CLIENTS
DEVICE_BASE = DEVICE

# Initialize WandB for FedAvg
wandb.init(project="fed-audit-gan", name=f"FedAvg_baseline_clients{N_CLIENTS_BASE}", config={
    "method": "FedAvg",
    "n_rounds": N_ROUNDS_BASE, 
    "n_clients": N_CLIENTS_BASE,
    "gamma": 0,  # No fairness weighting
    "device": DEVICE_BASE
})

# Use same data partition for fair comparison
transform_base = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
train_data_base = datasets.MNIST('./data', train=True, download=True, transform=transform_base)
test_data_base = datasets.MNIST('./data', train=False, download=True, transform=transform_base)

# Same partition function
def partition_data_base(dataset, n):
    idx = np.argsort([dataset[i][1] for i in range(len(dataset))])
    shards = np.array_split(idx, n * 2)
    np.random.seed(42)  # Same seed for reproducibility
    np.random.shuffle(shards)
    return [np.concatenate([shards[2*i], shards[2*i+1]]) for i in range(n)]

client_idx_base = partition_data_base(train_data_base, N_CLIENTS_BASE)
test_loader_base = DataLoader(test_data_base, batch_size=64)
val_loader_base = DataLoader(Subset(train_data_base, np.random.choice(len(train_data_base), 1000)), batch_size=32)

# Initialize model
global_model_fedavg = CNN_Baseline().to(DEVICE_BASE)

# History for FedAvg
history_fedavg = {'acc': [], 'bias': []}

print("=" * 60)
print("BASELINE: Standard FedAvg (Uniform Weights)")
print("=" * 60)
print(f"Rounds: {N_ROUNDS_BASE}, Clients: {N_CLIENTS_BASE}")
print("Aggregation: α_i = 1/N (equal weights for all clients)")
print("=" * 60)

# Reuse Generator for bias measurement (same as Fed-Audit-GAN)
G_base = FairnessGenerator()
D_base = Discriminator()
G_base, D_base = train_gan(G_base, D_base, global_model_fedavg, val_loader_base, epochs=N_GAN_EPOCHS, device=DEVICE_BASE)
G_base.eval()
with torch.no_grad():
    z_base = torch.randn(N_PROBES, G_base.latent_dim, device=DEVICE_BASE)
    lbl_base = torch.randint(0, 10, (N_PROBES,), device=DEVICE_BASE)
    x_base, xp_base = G_base(z_base, lbl_base)

for r in range(N_ROUNDS_BASE):
    print(f"\n=== FedAvg Round {r+1}/{N_ROUNDS_BASE} ===")
    
    # Phase 1: Client training (same as Fed-Audit-GAN)
    updates_base = []
    for c in tqdm(range(N_CLIENTS_BASE), desc="Client Training"):
        loader = DataLoader(Subset(train_data_base, client_idx_base[c]), batch_size=32, shuffle=True)
        local = copy.deepcopy(global_model_fedavg)
        before = copy.deepcopy(global_model_fedavg.state_dict())
        opt = optim.SGD(local.parameters(), lr=0.01)
        local.train()
        for _ in range(3):
            for d, t in loader:
                opt.zero_grad()
                F.cross_entropy(local(d.to(DEVICE_BASE)), t.to(DEVICE_BASE)).backward()
                opt.step()
        updates_base.append({k: local.state_dict()[k] - before[k] for k in before})
    
    # Measure bias for comparison
    B_base_fedavg = compute_bias_baseline(global_model_fedavg, x_base, xp_base, DEVICE_BASE)
    print(f"  Bias (B_base): {B_base_fedavg:.4f}")
    
    # FedAvg Aggregation: UNIFORM WEIGHTS (α_i = 1/N)
    alpha_uniform = [1.0 / N_CLIENTS_BASE] * N_CLIENTS_BASE
    print(f"  FedAvg Weights: {[f'{a:.3f}' for a in alpha_uniform]}")
    
    # Apply uniform aggregation
    sd = global_model_fedavg.state_dict()
    for k in sd:
        sd[k] = sd[k] + sum(a * u[k] for a, u in zip(alpha_uniform, updates_base))
    global_model_fedavg.load_state_dict(sd)
    
    # Evaluate
    acc_fedavg = evaluate_baseline(global_model_fedavg, test_loader_base, DEVICE_BASE)
    history_fedavg['acc'].append(acc_fedavg)
    history_fedavg['bias'].append(B_base_fedavg)
    
    wandb.log({'round': r+1, 'accuracy': acc_fedavg, 'bias': B_base_fedavg})
    print(f"  Accuracy: {acc_fedavg:.2f}%")

print(f"\nFedAvg Final Accuracy: {history_fedavg['acc'][-1]:.2f}%")
print(f"FedAvg Final Bias: {history_fedavg['bias'][-1]:.4f}")
wandb.finish()

In [ ]:
# ============================================================
# COMPARISON: Fed-Audit-GAN vs FedAvg
# Additional Metrics: JFI, Variance, Max-Min Gap
# ============================================================
import matplotlib.pyplot as plt
import numpy as np

# ============================================================
# FAIRNESS METRICS CALCULATION
# ============================================================

def calculate_jfi(weights):
    """
    Jain's Fairness Index (JFI)
    Formula: JFI = (Σ α_i)² / (N × Σ α_i²)
    Range: 1/N (worst) to 1 (best/equal)
    """
    weights = np.array(weights)
    n = len(weights)
    numerator = np.sum(weights) ** 2
    denominator = n * np.sum(weights ** 2)
    return numerator / denominator if denominator > 0 else 0

def calculate_variance(weights):
    """
    Variance of aggregation weights
    Lower = more equal distribution
    """
    return np.var(weights)

def calculate_max_min_gap(weights):
    """
    Max-Min Gap
    Difference between highest and lowest weight
    Lower = more equal distribution
    """
    return np.max(weights) - np.min(weights)

def calculate_coefficient_of_variation(weights):
    """
    Coefficient of Variation (CV)
    CV = std / mean
    Lower = more equal distribution
    """
    mean = np.mean(weights)
    std = np.std(weights)
    return std / mean if mean > 0 else 0

# Calculate metrics for Fed-Audit-GAN (using last round's alpha weights)
# We need to recalculate the final alpha weights
S_final = []
G_final = FairnessGenerator()
D_final = Discriminator()
G_final, D_final = train_gan(G_final, D_final, global_model, val_loader, epochs=5, device=DEVICE)
G_final.eval()
with torch.no_grad():
    z_f = torch.randn(N_PROBES, G_final.latent_dim, device=DEVICE)
    lbl_f = torch.randint(0, 10, (N_PROBES,), device=DEVICE)
    x_f, xp_f = G_final(z_f, lbl_f)

B_base_final = compute_bias(global_model, x_f, xp_f, DEVICE)
# Simulate client updates for final metrics
for c in range(N_CLIENTS):
    loader = DataLoader(Subset(train_data, client_idx[c]), batch_size=32, shuffle=True)
    local = copy.deepcopy(global_model)
    before = copy.deepcopy(global_model.state_dict())
    opt = optim.SGD(local.parameters(), lr=0.01)
    local.train()
    for _ in range(1):  # Quick update
        for d, t in loader:
            opt.zero_grad()
            F.cross_entropy(local(d.to(DEVICE)), t.to(DEVICE)).backward()
            opt.step()
            break
    upd = {k: local.state_dict()[k] - before[k] for k in before}
    hyp = copy.deepcopy(global_model)
    sd = hyp.state_dict()
    for k in sd: sd[k] = sd[k] + upd[k]
    hyp.load_state_dict(sd)
    B_i = compute_bias(hyp, x_f, xp_f, DEVICE)
    S_final.append(B_base_final - B_i)

# Fed-Audit-GAN weights (fairness-aware)
alpha_fedaudit = F.softmax(torch.tensor(S_final) * GAMMA, dim=0).numpy()

# FedAvg weights (uniform)
alpha_fedavg_uniform = np.array([1.0 / N_CLIENTS] * N_CLIENTS)

# Calculate all metrics
metrics_fedaudit = {
    'JFI': calculate_jfi(alpha_fedaudit),
    'Variance': calculate_variance(alpha_fedaudit),
    'Max-Min Gap': calculate_max_min_gap(alpha_fedaudit),
    'CV': calculate_coefficient_of_variation(alpha_fedaudit)
}

metrics_fedavg = {
    'JFI': calculate_jfi(alpha_fedavg_uniform),
    'Variance': calculate_variance(alpha_fedavg_uniform),
    'Max-Min Gap': calculate_max_min_gap(alpha_fedavg_uniform),
    'CV': calculate_coefficient_of_variation(alpha_fedavg_uniform)
}

print("\n" + "=" * 70)
print("📊 FAIRNESS METRICS COMPARISON")
print("=" * 70)
print(f"\n{'Metric':<25} {'Fed-Audit-GAN':<20} {'FedAvg':<20} {'Interpretation'}")
print("-" * 90)
print(f"{'JFI (Jain Index)':<25} {metrics_fedaudit['JFI']:.6f}{'':<10} {metrics_fedavg['JFI']:.6f}{'':<10} Higher = More Equal")
print(f"{'Variance':<25} {metrics_fedaudit['Variance']:.6f}{'':<10} {metrics_fedavg['Variance']:.6f}{'':<10} Lower = More Equal")
print(f"{'Max-Min Gap':<25} {metrics_fedaudit['Max-Min Gap']:.6f}{'':<10} {metrics_fedavg['Max-Min Gap']:.6f}{'':<10} Lower = More Equal")
print(f"{'Coeff. of Variation':<25} {metrics_fedaudit['CV']:.6f}{'':<10} {metrics_fedavg['CV']:.6f}{'':<10} Lower = More Equal")
print("=" * 70)

print(f"\n📈 Aggregation Weights:")
print(f"   Fed-Audit-GAN: {[f'{a:.4f}' for a in alpha_fedaudit]}")
print(f"   FedAvg:        {[f'{a:.4f}' for a in alpha_fedavg_uniform]}")

# ============================================================
# VISUALIZATION
# ============================================================

fig, axes = plt.subplots(2, 3, figsize=(18, 10))

rounds = range(1, len(history['acc']) + 1)

# Plot 1: Accuracy Comparison
axes[0, 0].plot(rounds, history['acc'], 'b-o', label='Fed-Audit-GAN (γ=2.0)', linewidth=2, markersize=8)
axes[0, 0].plot(rounds, history_fedavg['acc'], 'r--s', label='FedAvg (γ=0)', linewidth=2, markersize=8)
axes[0, 0].set_xlabel('Round', fontsize=12)
axes[0, 0].set_ylabel('Accuracy (%)', fontsize=12)
axes[0, 0].set_title('Test Accuracy Comparison', fontsize=14, fontweight='bold')
axes[0, 0].legend(fontsize=11)
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Bias Comparison
axes[0, 1].plot(rounds, history['bias'], 'b-o', label='Fed-Audit-GAN (γ=2.0)', linewidth=2, markersize=8)
axes[0, 1].plot(rounds, history_fedavg['bias'], 'r--s', label='FedAvg (γ=0)', linewidth=2, markersize=8)
axes[0, 1].set_xlabel('Round', fontsize=12)
axes[0, 1].set_ylabel('Bias (B_base)', fontsize=12)
axes[0, 1].set_title('Bias Comparison', fontsize=14, fontweight='bold')
axes[0, 1].legend(fontsize=11)
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Aggregation Weights Bar Chart
x_clients = np.arange(N_CLIENTS)
width = 0.35
bars1 = axes[0, 2].bar(x_clients - width/2, alpha_fedaudit, width, label='Fed-Audit-GAN', color='#3498db', edgecolor='black')
bars2 = axes[0, 2].bar(x_clients + width/2, alpha_fedavg_uniform, width, label='FedAvg', color='#e74c3c', edgecolor='black')
axes[0, 2].set_xlabel('Client ID', fontsize=12)
axes[0, 2].set_ylabel('Aggregation Weight (α)', fontsize=12)
axes[0, 2].set_title('Client Aggregation Weights', fontsize=14, fontweight='bold')
axes[0, 2].set_xticks(x_clients)
axes[0, 2].legend(fontsize=11)
axes[0, 2].grid(True, alpha=0.3, axis='y')

# Plot 4: Final Accuracy Bar Chart
methods = ['Fed-Audit-GAN', 'FedAvg']
final_accs = [history['acc'][-1], history_fedavg['acc'][-1]]
colors = ['#2ecc71', '#e74c3c']
bars = axes[1, 0].bar(methods, final_accs, color=colors, edgecolor='black', linewidth=2)
axes[1, 0].set_ylabel('Final Accuracy (%)', fontsize=12)
axes[1, 0].set_title('Final Accuracy', fontsize=14, fontweight='bold')
axes[1, 0].set_ylim([min(final_accs) - 5, max(final_accs) + 5])
for bar, acc in zip(bars, final_accs):
    axes[1, 0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
                    f'{acc:.2f}%', ha='center', fontsize=12, fontweight='bold')

# Plot 5: JFI Comparison
jfi_values = [metrics_fedaudit['JFI'], metrics_fedavg['JFI']]
bars = axes[1, 1].bar(methods, jfi_values, color=colors, edgecolor='black', linewidth=2)
axes[1, 1].set_ylabel('JFI Score', fontsize=12)
axes[1, 1].set_title('Jain\'s Fairness Index\n(1.0 = Perfect Equality)', fontsize=14, fontweight='bold')
axes[1, 1].set_ylim([0, 1.1])
axes[1, 1].axhline(y=1.0, color='green', linestyle='--', label='Perfect JFI', linewidth=2)
for bar, jfi in zip(bars, jfi_values):
    axes[1, 1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
                    f'{jfi:.4f}', ha='center', fontsize=12, fontweight='bold')

# Plot 6: All Fairness Metrics Comparison
metric_names = ['Variance\n(×100)', 'Max-Min\nGap', 'CV']
fedaudit_vals = [metrics_fedaudit['Variance']*100, metrics_fedaudit['Max-Min Gap'], metrics_fedaudit['CV']]
fedavg_vals = [metrics_fedavg['Variance']*100, metrics_fedavg['Max-Min Gap'], metrics_fedavg['CV']]
x_metrics = np.arange(len(metric_names))
width = 0.35
bars1 = axes[1, 2].bar(x_metrics - width/2, fedaudit_vals, width, label='Fed-Audit-GAN', color='#3498db', edgecolor='black')
bars2 = axes[1, 2].bar(x_metrics + width/2, fedavg_vals, width, label='FedAvg', color='#e74c3c', edgecolor='black')
axes[1, 2].set_ylabel('Value', fontsize=12)
axes[1, 2].set_title('Fairness Metrics\n(Lower = More Equal)', fontsize=14, fontweight='bold')
axes[1, 2].set_xticks(x_metrics)
axes[1, 2].set_xticklabels(metric_names)
axes[1, 2].legend(fontsize=11)
axes[1, 2].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('comparison_with_fairness_metrics.png', dpi=150, bbox_inches='tight')
plt.show()

# ============================================================
# FINAL SUMMARY TABLE
# ============================================================
print("\n" + "=" * 80)
print("📊 COMPLETE COMPARISON SUMMARY")
print("=" * 80)
print(f"\n{'METRIC':<30} {'FED-AUDIT-GAN':<20} {'FEDAVG':<20} {'WINNER'}")
print("-" * 80)

# Accuracy
acc_winner = "Fed-Audit-GAN" if history['acc'][-1] >= history_fedavg['acc'][-1] else "FedAvg"
print(f"{'Final Accuracy (%)':<30} {history['acc'][-1]:<20.2f} {history_fedavg['acc'][-1]:<20.2f} {acc_winner}")

# Bias
bias_winner = "Fed-Audit-GAN" if history['bias'][-1] <= history_fedavg['bias'][-1] else "FedAvg"
print(f"{'Final Bias (Lower=Better)':<30} {history['bias'][-1]:<20.6f} {history_fedavg['bias'][-1]:<20.6f} {bias_winner}")

# JFI
jfi_winner = "FedAvg" if metrics_fedavg['JFI'] >= metrics_fedaudit['JFI'] else "Fed-Audit-GAN"
print(f"{'JFI (Higher=More Equal)':<30} {metrics_fedaudit['JFI']:<20.6f} {metrics_fedavg['JFI']:<20.6f} {jfi_winner}")

# Variance
var_winner = "FedAvg" if metrics_fedavg['Variance'] <= metrics_fedaudit['Variance'] else "Fed-Audit-GAN"
print(f"{'Variance (Lower=More Equal)':<30} {metrics_fedaudit['Variance']:<20.6f} {metrics_fedavg['Variance']:<20.6f} {var_winner}")

# Max-Min Gap
gap_winner = "FedAvg" if metrics_fedavg['Max-Min Gap'] <= metrics_fedaudit['Max-Min Gap'] else "Fed-Audit-GAN"
print(f"{'Max-Min Gap (Lower=Better)':<30} {metrics_fedaudit['Max-Min Gap']:<20.6f} {metrics_fedavg['Max-Min Gap']:<20.6f} {gap_winner}")

print("=" * 80)

print(f"""
📝 INTERPRETATION:
   
   1. JFI (Jain's Fairness Index):
      Formula: JFI = (Σαᵢ)² / (N × Σαᵢ²)
      Range: 1/N (one client dominates) to 1.0 (perfectly equal)
      FedAvg always has JFI = 1.0 (uniform weights)
      Fed-Audit-GAN has JFI < 1.0 (intentionally favors fair clients)
   
   2. Variance:
      Measures spread of weights around mean
      FedAvg: Variance = 0 (all weights identical)
      Fed-Audit-GAN: Variance > 0 (weights differ by fairness)
   
   3. Max-Min Gap:
      Difference between highest and lowest weight
      FedAvg: Gap = 0 (all clients equal)
      Fed-Audit-GAN: Gap > 0 (fair clients get higher weight)
   
   ⚡ KEY INSIGHT:
   Fed-Audit-GAN intentionally has "worse" JFI/Variance/Gap metrics
   because it PURPOSEFULLY gives more weight to fairer clients!
   This is the trade-off: weight equality vs fairness-awareness.
""")

In [ ]:
# Step 5: Plot Results
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
ax1.plot(history['acc'], 'b-o')
ax1.set_xlabel('Round'); ax1.set_ylabel('Accuracy (%)'); ax1.set_title('Test Accuracy'); ax1.grid(True)
ax2.plot(history['bias'], 'r-s')
ax2.set_xlabel('Round'); ax2.set_ylabel('Bias'); ax2.set_title('Baseline Bias'); ax2.grid(True)
plt.tight_layout()
plt.savefig('results.png', dpi=150)
plt.show()
print("Check your WandB dashboard: https://wandb.ai")